In [1]:
# ipcluster start -n 8 --engines=MPIEngineSetLauncher
import ipyparallel as ipp
client = ipp.Client(profile="default")
%autopx

%autopx enabled


/home/diego/Myopt/Miniconda/miniconda3/envs/UIBCDFLab_dev/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@nauta')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [7]:
import sys
sys.path.append('/home/diego/Projects/Pynterpred_devel/')
import time
import pynterpred as pnt
import numpy as np
from matplotlib import pyplot as plt

In [4]:
from mpi4py import MPI
comm = MPI.COMM_WORLD

In [5]:
Barnase     = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/Barnase-Barstar/pdbs/Barnase.pdb'
Barstar     = '/home/diego/Projects/Pynterpred_devel/pynterpred/testsystems/Barnase-Barstar/pdbs/Barstar.pdb'
force_field = 'amber14-all.xml'
pH          = 7.0

In [12]:
receptor_pdb_file = Barnase
ligand_pdb_file = Barstar
forcefield = force_field
pH = pH
delta_x = 1.0
nside = 3
mpi_comm = comm
verbose = True

###

i_am_logger_out = False
if verbose:
    print('va',flush=True)
    if mpi_comm is None:
        i_am_logger_out = True
    elif (mpi_comm.Get_rank() == 0):
        i_am_logger_out = True

sys.stdout.flush()
        
if i_am_logger_out:
    print("Setting up the mechanical molecular context... ", end="", flush=True)
    time_start = time.time()

sys.stdout.flush()
comm.Barrier()
tmp_receptor = pnt.Receptor(receptor_pdb_file,forcefield,pH,mpi_comm)
tmp_ligand   = pnt.Ligand(ligand_pdb_file,forcefield,pH,mpi_comm)
tmp_context  = pnt.MMContext(tmp_receptor,tmp_ligand)

if i_am_logger_out:
    print(time.time()-time_start,'secs')
    time_start = time.time()
    print("Setting up the evaluation region... ", end="", flush=True)

tmp_region   = pnt.Region(tmp_receptor, tmp_ligand, delta_x=delta_x, nside=nside)
tmp_docking  = pnt.Docker(tmp_context,tmp_region,mpi_comm)

if i_am_logger_out:
    print(time.time()-time_start,'secs')
    time_start = time.time()
    print("Potential Energy evaluation of", tmp_region.num_centers*tmp_region.num_rotations , "different relative orientations started...")
tmp_docking.evaluation()

if i_am_logger_out:
    print(time.time()-time_start,'secs')
    time_start = time.time()
    print("Done")
    print("Complex at infinite distance with Potential Energy:", tmp_docking.potential_energies.min())
    print("Best orientation evaluated with Potential Energy:", tmp_docking.potential_energies.min())

del(tmp_receptor, tmp_ligand, tmp_context, tmp_region)




[stdout:0] 
va
Setting up the mechanical molecular context... 5.778419494628906 secs
Setting up the evaluation region... 4.805249452590942 secs
Potential Energy evaluation of 20736 different relative orientations started...
15.57195234298706 secs
Done
Complex at infinite distance with Potential Energy: 5581.529296875
Best orientation evaluated with Potential Energy: 5581.529296875
[stdout:1] va
[stdout:2] va
[stdout:3] va
[stdout:4] va
[stdout:5] va
[stdout:6] va
[stdout:7] va


In [ ]:
docking = pnt.predict(Barnase, Barstar, force_field, pH, delta_x = 1.0, nside = 3, mpi_comm = comm, verbose = True)

In [ ]:
docking = pnt.pynterpred(Barnase, Barstar, force_field, pH, delta_x = 0.5, nside = 5, mpi_comm = comm)
xx1, pes1 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
docking = pnt.pynterpred(Barnase, Barnase, force_field, pH, delta_x = 0.5, nside = 5, mpi_comm = comm)
xx2, pes2 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
docking = pnt.pynterpred(Barstar, Barstar, force_field, pH, delta_x = 0.5, nside = 5, mpi_comm = comm)
xx3, pes3 = docking.get_potential_energy_1D_landscape()
del(docking)

In [ ]:
plt.scatter(xx1,pes1,0.1,label="Bb")
plt.scatter(xx2,pes2,0.1,label="BB")
plt.scatter(xx3,pes3,0.1,label="bb")
plt.legend()

plt.xlim([-10000,10000])
plt.ylim([-2000,4000])
